In [0]:
%sql
-- Query 1
-- TABLE status_bronze -- Fill in code to create or refresh a streaming table
-- AS
-- SELECT
-- current_timestamp() AS processing_time, *
-- FROM cloud_files("$(source)/status", "json");

CREATE OR REFRESH STREAMING TABLE status_bronze 
AS
SELECT current_timestamp() AS processing_time, *
FROM cloud_files("${source}/status", "json");

-- Query 2
-- TABLE status_silver -- Fill in code to create or refresh a streaming table
-- (CONSTRAINT valid_timestamp EXPECT (status_timestamp > 1640600000) ON VIOLATION DROP ROW)
-- AS
-- SELECT * EXCEPT (_rescued_data)
-- FROM status _bronze; -- Fix the FROM clause to reference a the status_bronze streaming table

CREATE OR REFRESH STREAMING TABLE status_silver 
(CONSTRAINT valid_timestamp EXPECT (status_timestamp > 1640600000) ON VIOLATION DROP ROW)
AS
SELECT * EXCEPT (_rescued_data)
FROM STREAM(LIVE.status_bronze);

-- Query 3
-- VIEW email_updates -- Fill in code to create or refresh a streaming materialized view
-- AS
-- SELECT a.*, b.email
-- FROM status_silver a -- Fix the FROM clause to reference the status silver streaming table
-- INNER JOIN LIVE.subscribed_order_emails_v b
-- ON a.order_id = b.order_id;

CREATE OR REFRESH MATERIALIZED VIEW email_updates
AS
SELECT a.*, b.email
FROM LIVE.status_silver a
INNER JOIN LIVE.subscribed_order_emails_v b
ON a.order_id = b.order_id;